In [ ]:
pip install tensorflow

In [51]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [52]:
tf.test.is_gpu_available()

False

In [53]:
batchsize=8

In [54]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2, zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)

In [55]:
train_data= train_datagen.flow_from_directory(r'C:\Users\banda\OneDrive\Desktop\Project dataset\mrlEyes_2018_01\Prepared_Data\train',
                                             target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='training')

validation_data= train_datagen.flow_from_directory(r'C:\Users\banda\OneDrive\Desktop\Project dataset\mrlEyes_2018_01\Prepared_Data\train',
                                             target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64645 images belonging to 2 classes.
Found 16160 images belonging to 2 classes.


In [56]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\banda\OneDrive\Desktop\Project dataset\mrlEyes_2018_01\Prepared_Data\train',
                                             target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 80805 images belonging to 2 classes.


In [57]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3), batch_size=8))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation= 'softmax')(hmodel)


model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [58]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(8, 80, 80, 3)]             0         []                            
                                                                                                  
 conv2d_376 (Conv2D)         (8, 39, 39, 32)              864       ['input_6[0][0]']             
                                                                                                  
 batch_normalization_376 (B  (8, 39, 39, 32)              96        ['conv2d_376[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_376 (Activation  (8, 39, 39, 32)              0         ['batch_normalization_37

In [59]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [60]:
checkpoint = ModelCheckpoint(r'C:\Users\banda\OneDrive\Desktop\Project dataset\models\model.h5', 
                             monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose=3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [62]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                   epochs=5)

Epoch 1/5


C:\Users\banda\AppData\Local\Temp\ipykernel_6008\719301789.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


8080/8080 [==============================] - ETA: 0s - loss: 0.1641 - accuracy: 0.9369
Epoch 1: val_loss did not improve from 0.18771
8080/8080 [==============================] - 349s 43ms/step - loss: 0.1641 - accuracy: 0.9369 - val_loss: 0.2083 - val_accuracy: 0.9085 - lr: 0.0010
Epoch 2/5
8080/8080 [==============================] - ETA: 0s - loss: 0.1611 - accuracy: 0.9375
Epoch 2: val_loss did not improve from 0.18771
8080/8080 [==============================] - 373s 46ms/step - loss: 0.1611 - accuracy: 0.9375 - val_loss: 0.2005 - val_accuracy: 0.9178 - lr: 0.0010
Epoch 3/5
8079/8080 [============================>.] - ETA: 0s - loss: 0.1581 - accuracy: 0.9395
Epoch 3: val_loss did not improve from 0.18771
8080/8080 [==============================] - 392s 48ms/step - loss: 0.1581 - accuracy: 0.9395 - val_loss: 0.2027 - val_accuracy: 0.9155 - lr: 0.0010
Epoch 4/5
8079/8080 [============================>.] - ETA: 0s - loss: 0.1575 - accuracy: 0.9395
Epoch 4: val_loss did not improve 

In [ ]:
# Model Evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\banda\AppData\Local\Temp\ipykernel_6008\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


In [ ]:
acc_tr, loss_vr = model.evaluate_generator(validation_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_test, loss_tr = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)